In [65]:
# Inital state probability matrix, given
PI = [0.5,0.5]
# PI = {'Healthy': 0.5, 'Depressed': 0.5}
# state transition probability matrix, given
A = [[0.8,0.2],[0.001,0.999]]
# A = {
#     'Healthy' : {'Healthy': 0.8, 'Depressed': 0.2},
#     'Depressed' : {'Healthy': 0.001, 'Depressed': 0.999},
# }
# Emission or observation probability matrix, given
B = [[0.5,0.1,0.3,0.1,0.0],[0.05,0.35,0.2,0.2,0.2]]
# B = {
#     'Healthy' : {'Movement': 0.5, 'Passive': 0.1, 'Active' : 0.3, 'Texting' : 0.1, 'Access' : 0.0},
#     'Depressed' : {'Movement': 0.05, 'Passive': 0.35, 'Active' : 0.2, 'Texting' : 0.2, 'Access' : 0.2},
# }

# state codes of the observed states, dummy
# Observables = [0,1,2,3,4]
# Observables = ["Movement","Passive","Active","Texting","Access"]
#Observables = ["Movement", "Texting", "Movement", "Texting"]
#states = ["Healthy","Depressed"]
states = [0, 1]
Observables = [0,3,0,3]
#Observables = [0,0,2]
#Observables = {"Movement":0, "Passive Social":1, "Active Social":2, "Texting":3, "Access Psych Site":4}

In [128]:
PI_2 = [0.5,0.5]
A_2 = [[0.7,0.3],[0.3,0.7]]
B_2 = [[0.9,0.1],[0.2,0.8]]
states = [0,1]
Observables_2 = [0,0]

In [129]:
forward(PI_2,A_2,B_2,Observables_2)

[[0.45, 0.1]]


array([[0.45  , 0.3105],
       [0.1   , 0.041 ]])

In [171]:
normalized, c = forward_normalized(PI_2,A_2,B_2,Observables_2)

{0: 0.45, 1: 0.1}
C val: 
2.844950213371266


In [172]:
c

[1.8181818181818181, 2.844950213371266]

In [153]:
normalized

array([[0.81818182, 0.88335704],
       [0.18181818, 0.11664296]])

In [147]:
evaluation_normalized(c)

-1.6433825684870378

In [137]:
backward(PI_2,A_2,B_2,Observables_2)

array([[0.69, 1.  ],
       [0.41, 1.  ]])

# Filtering

In [126]:
import numpy as np 
from numpy import matrix

def forward(PI, A, B, Observables):
    alpha_matrix = []
    curr = {}
    #nex = [0] * len(A)
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
    #alpha_matrix.append(curr)
    alpha_matrix.append(list(curr.values()))

    for t in range (1, len(Observables)):
        #nex = [0] * len(A)
        nex = {}
        for i in range (len(A)):
            #for j in range (len(A)):
                #nex = alpha_matrix[t-1][j]
            t_value = Observables[t]
            nex[i] =  sum(curr[j] * A[j][i] * B[i][t_value] for j in range(len(A)))
                #print(nex)
            #nex[i] = nex[i] * B[i][t_value] 
        alpha_matrix.append(list(nex.values()))
        #alpha.insert(t,list(nex.values()))

        curr = nex.copy()
            
    alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix

def forward_normalized(PI, A, B, Observables):
    alpha = forward(PI, A, B, Observables)
    c = [0] * len(Observables)
    for i,i_value in enumerate(alpha):
        for j,j_value in enumerate(i_value):
            c[j] += i_value[j]
        c[j] = 1/c[j]
            
    
    for i in range(len(alpha)):
        for j in range(len(alpha[i])):
            alpha[i][j] = alpha[i][j] * c[j]
    
    return alpha, c

In [170]:
def forward_normalized(PI, A, B, Observables):
    alpha_matrix = []
    curr = {}
    c = {}
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
        
    #alpha_matrix.insert(list(curr.values()))
    alpha_matrix.append(list(curr.values()))
    c[0] = sum(alpha_matrix[0][i] for i in range(len(curr)))

    
    c[0] = 1/c[0]
    for i in range (0, len(A)):
        alpha_matrix[0][i] = c[0] * alpha_matrix[0][i]
    
    for t in range (1, len(Observables)):
        nex = {}
        c_val = 0
        for i in range (0, len(A)):
                #nex = alpha_matrix[t-1][j]
            t_value = Observables[t]
            nex[i] =  sum(curr[j] * A[j][i] * B[i][t_value] for j in range(len(A)))
        c[t] = sum(nex[k] for k in range(len(nex)))
        
            
        c[t] = 1/c[t]
        for i in range (0,len(A)):
            nex[i] = c[t] * nex[i]
        alpha_matrix.append(list(nex.values()))
        curr = nex
        
        
            
    alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix,list(c.values())

# Evaluation

In [141]:
def evaluation_unnormalized(alpha):
    probability = 0
    for i in range(len(alpha)):
        for j in range(0, len(alpha[i])):
             probability += alpha[i][j]        
    return probability

In [142]:
import math
def evaluation_normalized(c):
    probability = -1 * sum(math.log(i) for i in c)
    return probability

# Backward Probability

In [136]:
def backward(PI, A, B, Observables):
    
    beta = []
    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr

    #p_beta = sum(PI[l] * B[l][Observables[0]] * b_curr[l] for l in states)
    
    return np.transpose(beta) #no of initial states * length of observation sequence

In [138]:
def backward_normalized(PI, A, B, Observables):
    alpha,c = forward_normalized(PI, A, B, Observables)
    beta = backward(PI, A, B, Observables)


    for i in range(len(beta)):
        for j in range(len(beta[i])):
            beta[i][j] = beta[i][j] * (1/c[j])
            
    return beta

In [139]:
backward_normalized(PI, A, B, Observables)

array([[0.04125   , 0.04624375],
       [0.09618125, 0.04624375]])

# Smoothing by Forward-Backward Algorithm

In [88]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    matrix = []
    for t in range(len(alpha)):
        curr = [0] *len(A)
        for i in range(len(alpha[t])):
            sum_curr = 0
            for j in range(len(beta[t])):
                sum_curr += alpha[t][j] * beta[t][j]
            curr[i] = (alpha[t][i] * beta[t][i]) / sum_curr
        matrix.append(curr)  
                
    return matrix

In [89]:
smoothed_probability(alpha,beta,len(alpha), len(beta))

[[0.9962860631362895, 0.003713936863710455],
 [0.49618300053009035, 0.5038169994699098],
 [0.9035671451814147, 0.09643285481858543],
 [0.042700999388114926, 0.957299000611885]]